In [65]:
# auto reload libraries (you do need to re-import libraries if you make changes)
%load_ext autoreload
%autoreload 2

# base 
import pandas as pd
import numpy as np
from pprint import pprint
import os
from time import time 
# preprocessing 
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer, IterativeImputer,KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, ShuffleSplit, learning_curve
from sklearn.metrics import make_scorer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import FunctionTransformer
# variable importance
from sklearn.inspection import permutation_importance
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, SparsePCA

# visualize
import matplotlib.pyplot as plt
import matplotlib

# models
from sklearn.ensemble import RandomForestRegressor

# own defined functions/classes 
from preprocessing.preprocess_pipe import MakeLowerCase
from preprocessing.preprocess_pipe import HdResolutionCategorizer
from preprocessing.preprocess_pipe import StorageCategorizer
from preprocessing.preprocess_pipe import SsdCategorizer

from preprocessing.preprocess_pipe import print_missing
from preprocessing.preprocess_pipe import calculate_perf
from preprocessing.preprocess_pipe import custom_scoring_func

from modelling.weight_samples import weights_samples

from postprocessing.postprocessing import plot_predictions_results
from postprocessing.postprocessing import plot_residuals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# global parameters
pd.set_option('display.max_columns', 500)
# style for plotting
plt.style.use('ggplot')
# interactive plotting
%matplotlib widget
# run grid search
RUN_GRID_SEARCH = True
# set working directory
uppath = lambda _path, n: os.sep.join(_path.split(os.sep)[:-n])
__file__ = 'C:\\Users\\VictorGueorguiev\\OneDrive - Amplify Analytix\\Documents\\University\\Advanced Analytics for Business\\AAForB_Assignment2\\code\\Victor'
out = uppath(__file__, 2)
os.chdir(out)
print(os.getcwd())

C:\Users\VictorGueorguiev\OneDrive - Amplify Analytix\Documents\University\Advanced Analytics for Business\AAForB_Assignment2


## TODO

### PLEASE DEFINE FUNCTIONS IN SEPERATE FILES TO KEEP THE NOTEBOOK CLEAN

- Add custum scoring metric: **DONE**
- further integrate preprocessing steps from Arnaud **DONE**
- Integerate steps from Victor **Partly DONE**
- Look at comments of Victor for next steps
- Look at feature engineering ideas
- Integrate steps from Bram 
- Integrate custom scoring metric (also when performing the cross validation/grid search) **DONE**
- Make function that checkts whether prediction of the maximum price are >= predictions of the minimum price
- Investigate why there is such a huge difference between the performance on 'our test set' and the performance of submission test set
- Make function that saves trained models
- Try out other models (Boosting, Support Vector Machines, Penalized Linear models like lasso, stacked models ...)
- Implement better missing values imputations methods **DONE**
- Add some post processing visualizations:
    - feature importance plots
    - look at our predictions visually (do they make sense?)
    - look at the residuals
    - Have a look at this especially section 5 for model interpretability ideas
    https://christophm.github.io/interpretable-ml-book/pdp.html


# Step 1: import data and split in train and validation set
The validation set is more our own kind of test set

In [28]:
# read in trainig and validation data
# use the same data split as we did in R
df_all_train = pd.read_csv("data\\train.csv", sep=',')
df_test = pd.read_csv("data\\test.csv", sep=',')

print(f'Dimensions of all training data {df_all_train.shape}')
print(f'Dimension test data {df_test.shape}')

Dimensions of all training data (510, 22)
Dimension test data (222, 20)


In [29]:
# split in training and validation set
df_train = df_all_train.sample(frac=0.75, random_state=0, replace=False)
df_val = df_all_train.drop(df_train.index)

# reset index, if you don't resit missing rows get inserted in the pipeline
# see: https://stackoverflow.com/questions/31323499/sklearn-error-valueerror-input-contains-nan-infinity-or-a-value-too-large-for
df_train = df_train.reset_index().drop('index',axis=1)
df_val = df_val.reset_index().drop('index',axis=1)

In [41]:
df_all_train['cpu_frequency'] = df_all_train['cpu_details'].str.extract(r'\s*([0-9].[0-9][0-9])\s*[gG][hH][zZ]')
df_all_train['cpu_frequency2'] = df_all_train['cpu_details'].str.extract(r'\s*([0-9].[0-9])\s*[gG][hH][zZ]')

df_all_train['cpu_frequency'] = df_all_train['cpu_frequency'].replace(np.nan, '0')
df_all_train['cpu_frequency2'] = df_all_train['cpu_frequency2'].replace(np.nan, '0')

df_all_train['cpu_frequency'] = df_all_train['cpu_frequency'].astype(float)
df_all_train['cpu_frequency2'] = df_all_train['cpu_frequency2'].astype(float)
df_all_train['cpu_frequency'] = df_all_train['cpu_frequency'] + df_all_train['cpu_frequency2']

df_all_train = df_all_train.assign(cpu_frequency = [a if a > 0.0 else np.nan for a in df_all_train['cpu_frequency']])
df_all_train = df_all_train.drop(['cpu_frequency2'], axis = 1)
df_all_train['cpu_frequency'].isna().sum()

7

# Step 2: define pipeline

The pipeline should handle all steps performed on the data from data cleaning till the acutal model prediction. 
It should also be fairly easily to apply this to the test data. Therefore we make use of the scikit learn pipeline which allows all of this functionality. You can also implement your own `custom` transformations.

For more information on how to use and why using pipelines see:
 - https://medium.com/vickdata/a-simple-guide-to-scikit-learn-pipelines-4ac0d974bdcf 
 - scikit learn api on pipelines: https://scikit-learn.org/stable/modules/compose.html
 - Scikit learn api for more preprocessing steps: https://scikit-learn.org/stable/modules/preprocessing.html#non-linear-transformation

We make a difference between the pre processing steps for
- numerical features
- categorical features

It's important to note that if a feature is in the numerical feature pipeline it can't be in the categorical feature pipeline. Also if some preprocessing steps in the numerical feature pipeline need features from the categorical pipeline you will get an error. All preprocessing steps in each pipeline should be able to perform the transformation only with the categorical or numerical features. 

You can implement some custom transformation steps like 'MakeLowerCase' ==> see preprocessing.preprocess_pipe for more details or check this article  these two article (also gives some explanation on how to implement pipelines)

- https://gist.github.com/amberjrivera/8c5c145516f5a2e894681e16a8095b5c

In [47]:
#Numerical features to pass down the numerical pipeline 
numerical_features = ['screen_size', 'ram', 'weight']

#Transformative features for feature extraction
cat_to_num_features = ['cpu_details']

#Ordinal features to pass down he ordinal pipeline
ordinal_features = [ 'ssd','storage']

#Categrical features to pass down the categorical pipeline 
categorical_features = ['brand','base_name', 'screen_surface','touchscreen',
                        'cpu','detachable_keyboard',
                        'discrete_gpu','gpu', 'os', 'pixels_x','pixels_y']

# define all unique features
features = np.unique(cat_to_num_features).tolist() + np.unique(numerical_features).tolist() + np.unique(categorical_features).tolist() + np.unique(ordinal_features).tolist()

# target variables
target = ['min_price','max_price']

In [48]:
# train
X_train = df_train[features]
y_train = df_train[target]

# validation (this is kind our own test set)
X_val  = df_val[features]
y_val = df_val[target]

# train_validation (this is all training data we have) for fitting the model
X_all_train = df_all_train[features]
y_all_train = df_all_train[target]

# test
X_test = df_test[features]

In [31]:
from sklearn.base import BaseEstimator, TransformerMixin

class TsneReducer(BaseEstimator,TransformerMixin):
    
    """
    Categorize the SSD size into specific categories such as Small, Medium and Large, etc.
    """
    
    def __init__(self, drop_original_feature=True):
        
        self.drop_original_feature=drop_original_feature
        
    def fit(self, X, y=None):
        
        return self
        

    def transform(self, X):
        perplexity = 50
        t0 = time()
        tsne = TSNE(n_components=4, init='random',
                    method = 'exact',
                             random_state=0, perplexity=perplexity,
                   early_exaggeration = 50)
        X_tsne = tsne.fit_transform(X)                
        if self.drop_original_feature:
            return X_tsne
        else:
            return X

class CpuFrequencyExtractor(BaseEstimator,TransformerMixin):
    
    """
    Determine CPU frequency from cpu_details.
    """
    
    def __init__(self, drop_original_feature=True):
        
        self.drop_original_feature=drop_original_feature
        
    def fit(self, X, y=None):
        
        return self
        

    def transform(self, X):
              
        X['cpu_frequency'] = X['cpu_details'].str.extract(r'\s*([0-9].[0-9][0-9])\s*[gG][hH][zZ]')
        X['cpu_frequency2'] = X['cpu_details'].str.extract(r'\s*([0-9].[0-9])\s*[gG][hH][zZ]')

        X['cpu_frequency'] = X['cpu_frequency'].replace(np.nan, '0')
        X['cpu_frequency2'] = X['cpu_frequency2'].replace(np.nan, '0')

        X['cpu_frequency'] = X['cpu_frequency'].astype(float)
        X['cpu_frequency2'] = X['cpu_frequency2'].astype(float)
        X['cpu_frequency'] = X['cpu_frequency'] + X['cpu_frequency2']

        X = X.assign(cpu_frequency = [a if a > 0.0 else np.nan for a in X['cpu_frequency']])
        X = X.drop(['cpu_frequency2'], axis = 1)        
        if self.drop_original_feature:
            return X.drop(['cpu_details'], axis=1)
        else:
            return X  

    
class PCAReducer(BaseEstimator,TransformerMixin):
    
    """
    Categorize the SSD size into specific categories such as Small, Medium and Large, etc.
    """
    
    def __init__(self, drop_original_feature=True):
        
        self.drop_original_feature=drop_original_feature
        
    def fit(self, X, y=None):
        
        return self
        

    def transform(self, X):
        n_components = 15
        tsne = SparsePCA(n_components=n_components)
        X_pca = tsne.fit_transform(X.toarray())                
        if self.drop_original_feature:
            return X_pca
        else:
            return np.concatenate([X.toarray(), X_pca], axis = 1)  
        
class ScreenSizeFixer(BaseEstimator,TransformerMixin):
    
    """
    Fix discrepancies in screen_size variable.
    """
    
    def __init__(self, drop_original_feature=True):
        
        self.drop_original_feature=drop_original_feature
        
    def fit(self, X, y=None):
        
        return self
        

    def transform(self, X):
              
        X = X.replace(r'^\s*$', np.nan, regex=True)
        X['screen_surface'] = X['screen_surface'].replace('matte', 'Matte')
        X['screen_surface'] = X['screen_surface'].replace('glossy', 'Glossy')       
        if self.drop_original_feature:
            return X
        else:
            return X       
     

In [49]:
# you can add many more and 
# you can even define custom preprocessing steps like 'MakeLowerCase()'

cat_to_num_transformer = Pipeline(steps=[
    ('cpu_frequency', CpuFrequencyExtractor(drop_original_feature=True)),
    ('imputer', KNNImputer(n_neighbors=5, weights='uniform',metric='nan_euclidean')),
    ('scaler', RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25, 75.0))) 
])
# pipeline  uses only numerical features,
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('imputer', KNNImputer(n_neighbors=5, weights='uniform',metric='nan_euclidean')),
    #('imputer', IterativeImputer(missing_values=np.nan, random_state=0, n_nearest_features=None, sample_posterior=True)),
    #('transformation', PowerTransformer(method='yeo-johnson',standardize=False)),
     #Scale features using statistics that are robust to outliers.
    ('scaler', RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25, 75.0)))]) 

# pipeline use only categorical features
categorical_transformer = Pipeline(steps=[ 
    ('lowercase', MakeLowerCase()), # lower cases all columns containing strings
    ('screen_size', ScreenSizeFixer(drop_original_feature=False)),
    ('screen_resolution', HdResolutionCategorizer(drop_columns=['pixels_x', 'pixels_y'])),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# pipeline use only ordinal features
ordinal_transformer = Pipeline(steps=[ 
    ('lowercase', MakeLowerCase()), # lower cases all columns containing strings
    ('sd_category' ,SsdCategorizer(drop_original_feature=True)),
    ('storage_category', StorageCategorizer(drop_original_feature=True)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
    ('onehot', OrdinalEncoder())])

tsne_transformer = TsneReducer(drop_original_feature=True).transform
pca_transformer = PCAReducer(drop_original_feature=False).transform

# add both preprocessing pipelines in one pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat_to_num', cat_to_num_transformer, cat_to_num_features),
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)])

dimensionality_reducer = FunctionTransformer(
    pca_transformer
)

In [60]:
# this is how the preporcessed pipeline looks like (just to have an idea)
#pd.DataFrame.sparse.from_spmatrix(
X_train_transformed = preprocessor.fit_transform(X_train)
X_train_transformed = pd.DataFrame(dimensionality_reducer.fit_transform(X_train_transformed))
X_train_transformed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466
0,-0.666667,0.000000,0.666667,-0.020382,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,0.362851,0.319434,0.467801,-0.736466,0.833294,0.632419,0.755872,-0.805089,0.642137,-0.478661,0.480369,0.151015,-0.182921,-0.323601,0.469694
1,-0.444444,0.000000,0.666667,-0.351592,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [181]:
X_train_transformed.columns

RangeIndex(start=0, stop=163, step=1)

In [73]:
fig, ax = plt.subplots(1, 1, figsize=(9, 7))

perplexity = 50
t0 = time()
tsne = TSNE(n_components=2, init='random',
            method = 'exact',
            random_state=0, perplexity=perplexity,
            early_exaggeration = 50)
X_train_tsne = tsne.fit_transform(X_train_transformed)
#X_train_tsne = X_train_transformed
t1 = time()

cmap = plt.cm.rainbow
norm = matplotlib.colors.Normalize(vmin=1.5, vmax=4.5)

print(cmap(norm(y_train.min_price.values)))

print("circles, perplexity=%d in %.2g sec" % (perplexity, t1 - t0))
ax.set_title("Perplexity=%d" % perplexity)
ax.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c = y_train.min_price.values.tolist(), cmap='viridis')
#ax.scatter(X_train_tsne[green, 0], X_train_tsne[green, 1], c="g")
ax.axis('tight')
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # only needed for matplotlib < 3.1
fig.colorbar(sm)

plt.savefig('code\\Victor\\output\\figures\\random_forest\\TSNE.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[1.0000000e+00 1.2246468e-16 6.1232340e-17 1.0000000e+00]
 [1.0000000e+00 1.2246468e-16 6.1232340e-17 1.0000000e+00]
 [1.0000000e+00 1.2246468e-16 6.1232340e-17 1.0000000e+00]
 ...
 [1.0000000e+00 1.2246468e-16 6.1232340e-17 1.0000000e+00]
 [1.0000000e+00 1.2246468e-16 6.1232340e-17 1.0000000e+00]
 [1.0000000e+00 1.2246468e-16 6.1232340e-17 1.0000000e+00]]
circles, perplexity=50 in 9.1 sec


### Give more weights to higher priced computers

# Step 3: add models to pipeline

Possible to fit multiple target variabels, so you **don't** need to fit a different models for min. price and max. price

### A) Training and parameter tuning

##### 1) Automatic tuning via grid search

In [74]:
# define model: I just add some default parameters but you could
# also just write: RandomForestRegressor() since we will perform a grid search 
# to find good hyperparameter values
model = RandomForestRegressor(random_state=1, criterion='mse')

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(model.get_params())

# add to pipeline
pipeline = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('dimension_reducor', dimensionality_reducer),
                     ('regressor', model)])


# 1) min price
transformer_target = PowerTransformer(method='yeo-johnson',standardize=False)
scale_target = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(10.0, 90.0))
pipeline_y = Pipeline(memory=None,
              steps=[('transformer', transformer_target)])

pipeline_update = TransformedTargetRegressor(regressor=pipeline, 
                                         transformer=pipeline_y)

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 1,
 'verbose': 0,
 'warm_start': False}


In [184]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 4000, num = 20)]

# The number of features to consider when looking for the best split:
# - If “auto”, then max_features=n_features.
# - If “sqrt”, then max_features=sqrt(n_features).
# - If “log2”, then max_features=log2(n_features).
# - If None, then max_features=n_features
max_features = ['auto', 'sqrt','log2',None]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 150, num = 8)]
max_depth.append(None)

# The function to measure the quality of a split
criterion = ['mse','mae']

# Minimum number of samples required to split a node
# - If int, then consider min_samples_split as the minimum number.
# - If float, then min_samples_split is a fraction and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.
min_samples_split = [0.01, 0.025, 0.05, 0.1]
# Minimum number of samples required at each leaf node

# - If int, then consider min_samples_leaf as the minimum number.
# - If float, then min_samples_leaf is a fraction and ceil(min_samples_leaf * n_samples) are the minimum number of samples for each node.
min_samples_leaf = [0.01, 0.025, 0.05, 0.1]

# Create the random grid
random_grid = {
   'regressor__regressor__n_estimators': n_estimators,
   'regressor__regressor__max_features': max_features,
   'regressor__regressor__max_depth': max_depth,
   'regressor__regressor__criterion': criterion,
   'regressor__regressor__min_samples_split': min_samples_split,
   'regressor__regressor__min_samples_leaf': min_samples_leaf}

#### Helpful articles:
 - custom scoring metric: https://stackoverflow.com/questions/48468115/how-to-create-a-customized-scoring-function-in-scikit-learn-for-scoring-a-set-of
 - random parameter search: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
you can also use the traditional grid search in pyhton, but I prefer the randomizedSearch
 - Once the optimal parameters are found for the model, you don't need to run this again

In [142]:
# define random search (and narrow down time grid search)
if RUN_GRID_SEARCH:
    random_search = RandomizedSearchCV(
       estimator = pipeline_update, 
       param_distributions = random_grid, n_iter = 10,
       cv = 5, verbose=2, random_state=1, n_jobs = -1, refit=True,
       scoring=make_scorer(custom_scoring_func, greater_is_better=False)
    )


    # run grid search and refit with best hyper parameters
    weights_train_min_p =  weights_samples(df=y_train.iloc[:,0], order=0, plot_weights=False)
    random_search.fit(X_train, y_train)  
    print(random_search.best_params_)    
    print(random_search.best_score_)
    
# have look at the best hyperparameters and their respective performance (maybe also look at the sd)
pd.DataFrame(random_search.cv_results_).sort_values(
    by=['mean_test_score'],ascending=False).head(5)

Sum weights: 1.0
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  5.8min finished


{'regressor__regressor__n_estimators': 3178, 'regressor__regressor__min_samples_split': 0.1, 'regressor__regressor__min_samples_leaf': 0.01, 'regressor__regressor__max_features': 'log2', 'regressor__regressor__max_depth': None, 'regressor__regressor__criterion': 'mse'}
-935.7237488997696


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__regressor__n_estimators,param_regressor__regressor__min_samples_split,param_regressor__regressor__min_samples_leaf,param_regressor__regressor__max_features,param_regressor__regressor__max_depth,param_regressor__regressor__criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,12.737206,1.133072,3.184409,1.596584,3178,0.1,0.01,log2,None,mse,"{'regressor__regressor__n_estimators': 3178, '...",-1217.942750,-854.689410,-893.235842,-822.592394,-890.158348,-935.723749,143.453309,1
1,22.981410,1.092766,2.955600,1.633417,2563,0.01,0.01,log2,10,mae,"{'regressor__regressor__n_estimators': 2563, '...",-1225.166298,-849.791689,-886.063694,-828.689930,-889.549242,-935.852170,146.434743,2
3,20.118957,1.174587,2.995799,1.629387,1947,0.01,0.01,log2,90,mae,"{'regressor__regressor__n_estimators': 1947, '...",-1230.478493,-851.138211,-882.898750,-825.964127,-888.823833,-935.860683,149.045118,3
6,83.585819,2.460969,3.154800,1.594917,4000,0.1,0.01,auto,130,mae,"{'regressor__regressor__n_estimators': 4000, '...",-1430.502573,-819.341377,-835.464322,-1055.536973,-897.124992,-1007.594047,227.341711,4
7,7.317195,1.195701,2.736801,1.582222,100,0.01,0.01,None,150,mae,"{'regressor__regressor__n_estimators': 100, 'r...",-1429.524735,-829.990807,-857.794672,-1015.868437,-923.832758,-1011.402282,218.641339,5


##### Performance on validation data

##### 2) Manual parameters tuning

In [75]:
# train your final model on all data with best parameters 
model = RandomForestRegressor(
     criterion='mse',
     n_estimators=3200, 
     max_depth=200,
     max_features='log2',
     min_samples_split=0.1,
     min_samples_leaf=0.01,
     bootstrap=False,
     n_jobs=-1,
     random_state=1
)
# add to pipeline
pipeline = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('dimension_reducor', dimensionality_reducer),
                     ('regressor', model)],
              verbose=True)

# again add transformer for target variable
pipeline_update = TransformedTargetRegressor(regressor=pipeline, 
                                         transformer=pipeline_y)

# fit final model on all training data we have at hand
weights_train = weights_samples(y_train.iloc[:,1], order=2)
pipeline_update.fit(X_train, y_train)

Sum weights: 1.0
[Pipeline] ...... (step 1 of 3) Processing preprocessor, total=   0.1s
[Pipeline] . (step 2 of 3) Processing dimension_reducor, total=   3.2s
[Pipeline] ......... (step 3 of 3) Processing regressor, total=   3.7s


TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
                           regressor=Pipeline(memory=None,
                                              steps=[('preprocessor',
                                                      ColumnTransformer(n_jobs=None,
                                                                        remainder='drop',
                                                                        sparse_threshold=0.3,
                                                                        transformer_weights=None,
                                                                        transformers=[('cat_to_num',
                                                                                       Pipeline(memory=None,
                                                                                                steps=[('cpu_frequency',
                                                                                                

In [76]:
# performance on data where the model was fit one (should be very low)
pred_train = pipeline_update.predict(X_train)

# calculate performance
calculate_perf(y_train, pred_train)

{'minimum price': 282.7008610991055,
 'maximum price': 295.78876932138985,
 'total error': 578.4896304204954}

##### Performance on validation data

In [77]:
# performance on validation data
pred_val = pipeline_update.predict(X_val)

# calculate performance
calculate_perf(y_val, pred_val)

{'minimum price': 274.8883814477666,
 'maximum price': 287.05970524133966,
 'total error': 561.9480866891063}

In [35]:
# save predictions on validation data
# submission format
submission_format_validation = pd.DataFrame.from_dict(
 {'ID':df_val['id'].values,
 'MIN':pred_val[:,0],
 'MAX':pred_val[:,1]}).set_index('ID')

# write to csv
submission_format_validation.to_csv('code\\Victor\\output\\validation\\random_forest.csv' ,
                            header=True, index=True, sep=',')

### C) Post processing

In [36]:
fig, axs = plt.subplots(2, 2, figsize=(9, 7))
axs = np.ravel(axs)

# fitted against true predictions minimum price
plot_predictions_results(ax=axs[0], 
                        y_true=y_val.iloc[:,0], 
                        y_pred=pred_val[:,0], 
                        title="RF Min. Price Test Set", 
                        log_scale=True)

# fitted against true predictions maximum price
plot_predictions_results(ax=axs[1], 
                        y_true=y_val.iloc[:,1], 
                        y_pred=pred_val[:,1], 
                        title="RF Max. Price Test Set", 
                        log_scale=True)

# residuals minimum price
plot_residuals(ax=axs[2], 
               y_true=y_val.iloc[:,0], 
               y_pred=pred_val[:,0], 
               title="", 
               log_scale=False,
               order=1)

# residuals maximum price
plot_residuals(ax=axs[3], 
               y_true=y_val.iloc[:,1], 
               y_pred=pred_val[:,1], 
               title="", 
               log_scale=False,
               order=1)
fig.tight_layout()
plt.savefig('code\\Victor\\output\\figures\\random_forest\\fig1.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
plt.figure(figsize=(8,4))
plt.plot(pred_val[:,1] - pred_val[:,0], label="Predictions Test", linestyle="--")
plt.plot(y_val.iloc[:,1] - y_val.iloc[:,0], label="Truth Test", linestyle=':')
plt.xlabel("Observations")
plt.ylabel("Max. Price - Min. Price")
plt.legend(loc="upper right");
plt.savefig('code\\Victor\\output\\figures\\random_forest\\fig2.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
vip_val = permutation_importance(pipeline_update, X_val, y_val, n_repeats=10,
                                random_state=1, n_jobs=3)

sorted_idx = vip_val.importances_mean.argsort()
fig, ax = plt.subplots()
ax.boxplot(vip_val.importances[sorted_idx].T,
           vert=False, labels=X_val.columns[sorted_idx])
ax.set_title("Permutation Importances (Test set)")
fig.tight_layout()
plt.show()
plt.savefig('code\\Victor\\output\\figures\\random_forest\\fig3.png')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### D) Predictions test data

Refit on all training data (using the parameters found on the random search) and submit prediction

In [99]:
# train your final model on all data with best parameters 
model_final = RandomForestRegressor(
     criterion='mse',
     n_estimators=3200, 
     max_depth=10,
     max_features=None,
     min_samples_split=0.02,
     min_samples_leaf=0.02,
     bootstrap=False,
     n_jobs=-1,
     random_state=1
)
# add to pipeline
pipeline_final = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('dimension_reducor', dimensionality_reducer),
                     ('regressor', model_final)],
              verbose=True)

# again add transformer for target variable
pipeline_final = TransformedTargetRegressor(regressor=pipeline_final, 
                                         transformer=pipeline_y)

# fit final model on all training data we have at hand and give more weight to certain samples
weights_all_train = weights_samples(y_all_train.iloc[:,1], order=4)
pipeline_final = pipeline_final.fit(X_all_train, y_all_train, regressor__sample_weight=weights_all_train)

Sum weights: 1.0
[Pipeline] ...... (step 1 of 3) Processing preprocessor, total=   0.1s
[Pipeline] . (step 2 of 3) Processing dimension_reducor, total=   2.0s
[Pipeline] ......... (step 3 of 3) Processing regressor, total=   3.4s


In [100]:
# performance on all data where the model was fit one (should be very low)
calculate_perf(pipeline_final.predict(X_all_train), y_all_train)

{'minimum price': 252.09157850660142,
 'maximum price': 264.0082261103322,
 'total error': 516.0998046169336}

In [49]:
fig, axes = plt.subplots(3, 2, figsize=(10, 15))

title = "Learning Curves (Random Forest)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=50, test_size=0.2, random_state=0)

estimator = pipeline_final
plot_learning_curve(estimator, title, X_all_train, y_all_train, axes=axes[:, 0],
                    cv=cv, n_jobs=4)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<module 'matplotlib.pyplot' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\matplotlib\\pyplot.py'>

In [50]:
fig.tight_layout()
plt.show()
plt.savefig('code\\Victor\\output\\figures\\random_forest\\learning_curves.png')

In [42]:
# make predictions on test data
pred_test = pipeline_final.predict(X_test)

# submission format
submission_format = pd.DataFrame.from_dict(
 {'ID':df_test['id'].values,
 'MIN':pred_test[:,0],
 'MAX':pred_test[:,1]}).set_index('ID')

# write to csv
submission_format.to_csv('code\\Victor\\output\\submission\\random_forest.csv' ,
                            header=True, index=True, sep=',')